# Question 4 (20 pts) -- COMPLETE
By using a binary classifier (logistic regression or SVM) please implement in python the 2 different procedures
(one vs. one, one vs. other) to handle the problem of more than two different classes (use the iris dataset).
For reference, use the following link :
https://en.wikipedia.org/wiki/Multiclass_classification


From how I understood the clarifying of this question in lecuture on April 3rd we can use scklearn to implement one vs one, i.e. binary classification. However we need to implement our own one vs other classification. 

In [ ]:
from sklearn import datasets
from sklearn.decomposition import PCA

# import data
iris = datasets.load_iris()

In [ ]:
import pandas as pd
iris_X = iris.data
iris_y = iris.target
X = pd.DataFrame(iris_X, columns=iris.feature_names)
y = pd.DataFrame(iris_y, columns={"target"})
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

### One vs. Rest -- Logistic Regession

Inputs:
L, a learner (training algorithm for binary classifiers)
samples X
labels y where yi ∈ {1, … K} is the label for the sample Xi

Output:
a list of classifiers fk for k ∈ {1, …, K}

Procedure:
For each k in {1, …, K}
Construct a new label vector z where zi = yi if yi = k and zi = 0 otherwise
Apply L to X, z to obtain fk

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np 

In [ ]:
model0 = LogisticRegression() # setosa
model1 = LogisticRegression() # versicolor
model2 = LogisticRegression() # virginica

# split
X_train, X_test, y_train, y_test = train_test_split(iris_X, iris_y, test_size = 0.2)

In [ ]:
# for model 0
y_train00 = np.where(y_train == 0, 1, 0)
y_train01 = np.where(y_train == 1, 1, 0)
y_train02 = np.where(y_train == 2, 1, 0)

In [ ]:
model0.fit(X_train, y_train00)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
model1.fit(X_train, y_train01)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
model2.fit(X_train,y_train02)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# The label belongs in one
import random

y_pred = []
foo = [1,2,3]
for x in X_test:
   x = x.reshape(1, -1)
   z = model0.predict(x)
   y = 0
   if z!= 1:
     z = model1.predict(x)
     y = 1
     if z!=1:
       z = model2.predict(x)
       y = 2
       if z!= 1:
         y =random.choice(foo) # some error
   y_pred.append(y)

In [ ]:
# accuracy score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
print(accuracy_score(y_test, y_pred) )
print(precision_score(y_test, y_pred, average='weighted'))

0.6333333333333333
0.7703703703703703


Not a bad accuracy score for our simple one vs rest implementation but it could be better. It ranges between .63 and .83 depending on the split

### One vs One
* 0 vs 1
* 0 vs 2
* 1 vs 2

... I have a feeling this will look super similar

In [ ]:
# this is a horrible way to do it
ovo_01 = []
ovo_02 = []
ovo_12 = []

Xovo_01 = []
Xovo_02 = []
Xovo_12 = []
count = 0

for i in y_train:
  if i == 0 or i == 1:
    ovo_01.append(i)
    Xovo_01.append(X_train[count])
  if i == 0 or i == 2:
    ovo_02.append(i)
    Xovo_02.append(X_train[count])
  if i == 1 or i ==2:
    ovo_12.append(i)
    Xovo_12.append(X_train[count])
  count = count +1

 At this point all y and X values are the same lengths 
 

In [ ]:
ovo_model_01 = LogisticRegression()
ovo_model_02 = LogisticRegression()
ovo_model_12 = LogisticRegression()

In [ ]:
ovo_model_01.fit(Xovo_01, ovo_01)
ovo_model_02.fit(Xovo_02, ovo_02)
ovo_model_12.fit(Xovo_12, ovo_12)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
from scipy import stats
  
y_pred02 = []
foo = [0,1,2]

for x in X_test:
   x = x.reshape(1, -1)
   p1 = ovo_model_01.predict(x)
   p2 = ovo_model_02.predict(x) 
   p3 = ovo_model_12.predict(x)
   com = [p1,p2,p3]
   # this was overly complicated 
   y = stats.mode(com, axis = 0)
   y = y.mode
   if y == 0:
    y_pred02.append(0)
   if y == 1:
    y_pred02.append(1)
   if y == 2:
    y_pred02.append(2)

In [ ]:
accuracy_score(y_test, y_pred02) 

1.0

Thats not entierly good, might be over fitting, lets see precision

In [ ]:
precision_score(y_test, y_pred02, average='weighted')

1.0